In [3]:
import WDFunctions as w

take_data_from_url = input('''
Data will be taken from spreadsheets or downloaded from url.
Do you have the spreadsheets available? [Y/N]
''')

if take_data_from_url.lower() == 'y':
    import glob
    location = '*' # path name to directory of files, * for current directory
    files = glob.glob('*')
    
    file = [file for file in files if 'published' in file.lower()][0] #'publishedweek202020.xlsx'
    tabs = w.loadxlstabs(file)
    w.WeeklyDeathsByRegion(tabs)
    w.WeeklyDeathsByAgeSex(tabs)

    file = [file for file in files if 'lahb' in file] # 'lahbtablesweek20finalcodes.xlsx'
    if len(file) == 0:
        file = [file for file in files if 'la_hb' in file.lower()][0]
    else: file = file[0]
    reg_data = w.pd.read_excel(file, sheet_name='Registrations - All data', skiprows=3)
    occ_data = w.pd.read_excel(file, sheet_name='Occurrences - All data', skiprows=3)
    w.WeeklyDeathsByLA_HB(reg_data, occ_data, '2020')
    
    print('Transforms complete!')

else:
    print('Taking data from url download')
    
    from gssutils import *

    # get weekly deaths data from url -> by regions & by sex and age
    url = 'https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/weeklyprovisionalfiguresondeathsregisteredinenglandandwales'
    scraper = Scraper(url)

    # get required tabs -> passed to functions
    tabs = scraper.distributions[0].as_databaker()

    # get weekly deaths data from url -> by local authority & health board
    url = 'https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/causesofdeath/datasets/deathregistrationsandoccurrencesbylocalauthorityandhealthboard'
    scraper = Scraper(url)

    # import tabs separately -> registrations & occurrences -> to pass to function
    reg_data = scraper.distributions[0].as_pandas(sheet_name='Registrations - All data', skiprows=3)
    occ_data = scraper.distributions[0].as_pandas(sheet_name='Occurrences - All data', skiprows=3)

    # run functions
    w.WeeklyDeathsByRegion(tabs)
    w.WeeklyDeathsByAgeSex(tabs)
    w.WeeklyDeathsByLA_HB(reg_data, occ_data, '2020')
    
    print('Transforms complete!')


Data will be taken from spreadsheets or downloaded from url.
Do you have the spreadsheets available? [Y/N]
 y


Loading Published_Week_24_2020.xlsx which has size 194898 bytes
Table names: ['Analysis', 'Contents', 'Information', 'Terms and conditions', 'Weekly figures 2020', 'Covid-19 - Weekly registrations', 'Covid-19 - Weekly occurrences', 'UK - Covid-19 - Weekly reg', 'Covid-19 - E&W comparisons', 'Covid-19 - England comparisons', 'Covid-19 - Wales comparison', 'Covid-19 - Place of occurrence ', 'Related publications']
TIMEUNIT='Year'
TIMEUNIT='Year'
TIMEUNIT='Year'
Extracting data structured as v4.
Quick comparison check on v4-weekly-deaths-regional.csv
Difference between week numbers is not correct
Previous week goes up to 24
New week goes up to 24
New and previous v4 have same length, are week numbers the same
TIMEUNIT='Year'
TIMEUNIT='Year'
TIMEUNIT='Year'
Extracting data structured as v4.
Quick comparison check on v4-weekly-deaths-age-sex.csv
Difference between week numbers is not correct
Previous week goes up to 24
New week goes up to 24
New and previous v4 have same length, are week nu